Zonal statistics
================

Quite often you have a situtation when you want to summarize raster datasets based on vector geometries, such as calculating the average elevation of specific area.

[Rasterstats](https://github.com/perrygeo/python-rasterstats) is a Python module that does exactly that, easily.

- Let's start by reading the data:

In [ ]:
import rasterio
from rasterio.plot import show
from rasterstats import zonal_stats
import osmnx as ox
import geopandas as gpd
import os
import matplotlib.pyplot as plt
%matplotlib inline

# File path
data_dir = "L5_data"
dem_fp = os.path.join(data_dir, "Helsinki_DEM2x2m_Mosaic.tif")

# Read the Digital Elevation Model for Helsinki
dem = rasterio.open(dem_fp)
dem

Good, now our elevation data is in read mode.

Next, we want to calculate the elevation of two neighborhoods located in Helsinki, called `Kallio` and `Pihlajamäki`, and find out which one of them is higher based on the elevation data. We will use a package called [OSMnx](https://github.com/gboeing/osmnx) to fetch the data from OpenStreetMap for those areas.

- Specify place names for `Kallio` and `Pihlajamäki` that Nominatim can identify https://nominatim.openstreetmap.org/, and retrieve the 

In [ ]:
# Keywords for Kallio and Helsinki in such format that they can be found from OSM
kallio_q = "Kallio, Helsinki, Finland"
pihlajamaki_q = "Pihlajamäki, Malmi, Helsinki, Finland"

# Retrieve the geometries of those areas using osmnx
kallio = ox.gdf_from_place(kallio_q)
pihlajamaki = ox.gdf_from_place(pihlajamaki_q)

# Reproject to same coordinate system as the 
kallio = kallio.to_crs(crs=dem.crs)
pihlajamaki = pihlajamaki.to_crs(crs=dem.crs)

type(kallio)

As we can see, now we have retrieved data from OSMnx and they are stored as GeoDataFrames.

- Let's see how our datasets look by plotting the DEM and the regions on top of it

In [ ]:
# Plot the Polygons on top of the DEM
ax = kallio.plot(facecolor='None', edgecolor='red', linewidth=2)
ax = pihlajamaki.plot(ax=ax, facecolor='None', edgecolor='blue', linewidth=2)

# Plot DEM
show((dem, 1), ax=ax)

**Which one is higher? Kallio or Pihlajamäki? We can use zonal statistics to find out!**

- First we need to get the values of the dem as numpy array and the affine of the raster

In [ ]:
# Read the raster values
array = dem.read(1)

# Get the affine
affine = dem.transform

- Now we can calculate the zonal statistics by using the function `zonal_stats`.

In [ ]:
# Calculate zonal statistics for Kallio
zs_kallio = zonal_stats(kallio, array, affine=affine, stats=['min', 'max', 'mean', 'median', 'majority'])

# Calculate zonal statistics for Pihlajamäki
zs_pihla = zonal_stats(pihlajamaki, array, affine=affine, stats=['min', 'max', 'mean', 'median', 'majority'])


Okey. So what do we have now?

In [ ]:
print(zs_kallio)
print(zs_pihla)

Super! Now we can see that Pihlajamäki seems to be slightly higher compared to Kallio.